# Q Factor

## Reformat Data

**Import the two data tables.**
1. Table of all the relevant publications
2. List of all the SCOPUS_ID for authors of interest

In [5]:
import pandas as pd

# Load the DataFrame from an Excel file
df_all_works = pd.read_excel("Combined_Scopus_Published_Works.xlsx")
df_author_IDs = pd.read_excel("Scopus_IDs.xlsx")

print("There are",len(df_author_IDs),"authors.")

There are 1303 authors.


Clean up the dataframes by **removing duplicate rows**, and briefly view the data.

In [7]:
# Remove any duplicate rows
df_all_works = df_all_works.drop_duplicates()
df_author_IDs = df_author_IDs.drop_duplicates()

print("There are",len(df_author_IDs),"non-duplicate authors.")
print("\n",df_all_works.describe())

There are 1280 authors.

              Authors               Author full names  Author(s) ID    Title  \
count          73088                           73089         73074    73074   
unique         63657                           64328         63063    67400   
top     Pollock B.E.  Pollock, Bruce E. (7103208372)    7103208372  Preface   
freq             101                              91           101       52   

         Year                                       Source title  Volume  \
count   73075                                              73069   70187   
unique    110                                               4539    1090   
top      2021  International Journal of Radiation Oncology Bi...       7   
freq     4474                                               8055    1565   

        Issue  Art. No.  Page start  ...     ISSN             ISBN    CODEN  \
count   64682      8734       63958  ...    67930             9441    50941   
unique    917      6480        7958  ..

Create a new data table which takes all the authors' Scopus IDs, and lists all the publications and associated information for which they are listed.

In [9]:
#Convert SCOPUS_ID from integer to string type
df_author_IDs['SCOPUS_ID'] = df_author_IDs['SCOPUS_ID'].astype(str)

#Copy column that contains author IDs into a new column called 'All_Authors'
df_all_works['All_Authors'] = df_all_works['Author(s) ID']

#Split the 'Author(s) ID' column into separate rows in the all_works dataframe
df_all_works = df_all_works.assign(**{'Author(s) ID': df_all_works['Author(s) ID'].str.split('; ')}).explode('Author(s) ID')

#Match the SCOPUS_IDs from list with the Author_IDs in publication dataset to make a new dataset with merged data
merged_data = pd.merge(df_author_IDs, df_all_works, left_on='SCOPUS_ID', right_on='Author(s) ID', how='inner')

merged_data.to_csv('merged_dataframe.csv', index=False)
merged_data.describe()

,SCOPUS_ID,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,...,ISBN,CODEN,PubMed ID,Language of Original Document,Document Type,Publication Stage,Open Access,Source,EID,All_Authors
count,125743,125743,125743,125743,125742,125743,125738,121613,111522,15718,...,19626,89278,104792,125719,115708,125719,72035,105327,105327,125743
unique,1263,62709,63195,1263,64747,61,4344,1031,821,6367,...,7542,1808,49052,14,19,3,8236,1,61957,62248
top,35403900900,Hahn E.A.; Pugh S.L.; Lu H.L.; Vela A.M.; Gill...,"Hahn, Elizabeth A. (7202056032); Pugh, Stephan...",35403900900,Validation of Patient-Reported Outcomes in Pat...,2021,International Journal of Radiation Oncology Bi...,7,1,507,...,978-143771637-5,English,Article,English,Article,Final,All Open Access; Green Open Access,Scopus,2-s2.0-85192746169,7202056032; 55751695500; 26026423600; 57191992...
freq,1139,130,130,1139,130,9299,18546,2975,18400,62,...,138,20390,15984,105229,79586,104575,15807,105327,65,130


## Calculating Q factor